<a href="https://colab.research.google.com/github/nikhil-xyz/Natural_Language_Processing/blob/main/Fake_News_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/nikhil-xyz/Natural_Language_Processing.git

Cloning into 'Natural_Language_Processing'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 23 (delta 4), reused 14 (delta 3), pack-reused 0
Unpacking objects: 100% (23/23), done.


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/Natural_Language_Processing/data/Fake_News_Classifier/Fake_news.csv')

In [7]:
df = df.dropna()

In [9]:
X = df.drop('Label', axis=1)
y = df['Label']

In [12]:
import tensorflow as tf

In [13]:
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [14]:
vocab_size = 5000

In [15]:
messages = X.copy()
messages.reset_index(inplace=True)

In [16]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [47]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['Body'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [21]:
corpus[0]

'four way bob corker skewer donald trump'

In [48]:
onehot_rep = [one_hot(words, vocab_size) for words in corpus]

In [23]:
sentence_len = 2000

In [49]:
embedded_docs = pad_sequences(onehot_rep, padding='pre', maxlen=sentence_len)

In [26]:
len(embedded_docs)

3988

In [55]:
embedding_vector_features = 100
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features, input_length=sentence_len))
model.add(LSTM(200))
model.add(Dense(1, activation='sigmoid'))

In [56]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [57]:
import numpy as np
X = np.array(embedded_docs)
y = np.array(y)

In [44]:
X.shape

(3988, 20)

In [36]:
y.shape

(3988,)

In [58]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=5)

In [59]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10
100/100 [==============================] - 6s 49ms/step - loss: 0.2535 - accuracy: 0.9050 - val_loss: 0.1112 - val_accuracy: 0.9599
Epoch 2/10
100/100 [==============================] - 4s 45ms/step - loss: 0.0356 - accuracy: 0.9878 - val_loss: 0.0962 - val_accuracy: 0.9687
Epoch 3/10
100/100 [==============================] - 5s 45ms/step - loss: 0.0078 - accuracy: 0.9984 - val_loss: 0.1201 - val_accuracy: 0.9662
Epoch 4/10
100/100 [==============================] - 4s 45ms/step - loss: 5.0556e-04 - accuracy: 1.0000 - val_loss: 0.1589 - val_accuracy: 0.9624
Epoch 5/10
100/100 [==============================] - 4s 44ms/step - loss: 1.8440e-04 - accuracy: 1.0000 - val_loss: 0.2110 - val_accuracy: 0.9649
Epoch 6/10
100/100 [==============================] - 4s 43ms/step - loss: 1.3192e-04 - accuracy: 1.0000 - val_loss: 0.2246 - val_accuracy: 0.9637
Epoch 7/10
100/100 [==============================] - 5s 48ms/step - loss: 8.4182e-05 - accuracy: 1.0000 - val_loss: 0.2440 - val_